# Vectores de Lorentz, IDs de partículas PDG, agrupamiento de jets, ¡ay!

## Vectores de Lorentz

Siguiendo con la filosofía de "muchos paquetes pequeños", los vectores 2D/3D/Lorentz son manejados por un paquete llamado Vector. Aquí es donde puedes encontrar cálculos como `deltaR` y transformaciones de coordenadas.

In [ ]:
import vector

uno = vector.obj(px=1, py=0, pz=0)
dos = vector.obj(px=0, py=1, pz=1)

uno + dos

In [ ]:
uno.deltaR(dos)

In [ ]:
uno.to_rhophieta()

In [ ]:
dos.to_rhophieta()

Para integrarse con el resto del ecosistema, Vector debe ser una librería orientada a arreglos. Los arreglos de vectores 2D/3D/Lorentz se procesan en masa.

`MomentumNumpy2D`, `MomentumNumpy3D`, `MomentumNumpy4D` son subtipos de arreglos NumPy: los arreglos de NumPy pueden ser *convertidos* a estos tipos y obtener todas las funciones vectoriales.

In [ ]:
import skhep_testdata, uproot
import awkward as ak
import vector

tree = uproot.open(skhep_testdata.data_path("uproot-Zmumu.root"))["events"]

uno = ak.to_numpy(tree.arrays(filter_name=["E1", "p[xyz]1"]))
dos = ak.to_numpy(tree.arrays(filter_name=["E2", "p[xyz]2"]))

uno.dtype.names = ("E", "px", "py", "pz")
dos.dtype.names = ("E", "px", "py", "pz")

uno = uno.view(vector.MomentumNumpy4D)
dos = dos.view(vector.MomentumNumpy4D)

uno + dos

In [ ]:
uno.deltaR(dos)

In [ ]:
uno.to_rhophieta()

In [ ]:
dos.to_rhophieta()

Después de llamar a `vector.register_awkward()`, `"Momentum2D"`, `"Momentum3D"`, `"Momentum4D"` son nombres de registros que Awkward Array reconocerá para obtener todas las funciones vectoriales.

In [ ]:
vector.register_awkward()

tree = uproot.open(skhep_testdata.data_path("uproot-HZZ.root"))["events"]

array = tree.arrays(filter_name=["Muon_E", "Muon_P[xyz]"])

muones = ak.zip(
    {"px": array.Muon_Px, "py": array.Muon_Py, "pz": array.Muon_Pz, "E": array.Muon_E},
    with_name="Momentum4D",
)
mu1, mu2 = ak.unzip(ak.combinations(muones, 2))

mu1 + mu2

In [ ]:
mu1.deltaR(mu2)

In [ ]:
muones.to_rhophieta()

## Propiedades de partículas e identificadores PDG

La librería Particle proporciona todas las masas de partículas, anchos de decaimiento y más del PDG.
Además, contiene una serie de herramientas para consultar programáticamente las propiedades de las partículas y usar varios esquemas de identificación.

In [ ]:
import particle
from hepunits import GeV

particle.Particle.findall("pi")

In [ ]:
boson_z = particle.Particle.from_name("Z0")
boson_z.mass / GeV, boson_z.width / GeV

In [ ]:
print(boson_z.describe())

In [ ]:
particle.Particle.from_pdgid(111)

In [ ]:
particle.Particle.findall(
    lambda p: p.pdgid.is_meson and p.pdgid.has_strange and p.pdgid.has_charm
)

In [ ]:
print(particle.PDGID(211).info())

In [ ]:
pdgid = particle.Corsika7ID(11).to_pdgid()
particle.Particle.from_pdgid(pdgid)

# Agrupamiento de jets

En una colisión pp de alta energía, por ejemplo, se produce una lluvia de hadrones que se agrupan en `jets` de partículas, y este método/proceso se llama agrupamiento de jets (jet-clustering). El algoritmo de agrupamiento de jets anti-kt es uno de los algoritmos utilizados para combinar partículas/hadrones que están cerca entre sí en jets.

Algunas personas necesitan realizar agrupamiento de jets a nivel de análisis. El paquete fastjet hace posible hacerlo un array (Awkward) a la vez.

In [ ]:
import skhep_testdata, uproot
import awkward as ak
import particle
from hepunits import GeV
import vector

vector.register_awkward()

picodst = uproot.open(
    "https://pivarski-princeton.s3.amazonaws.com/pythia_ppZee_run17emb.picoDst.root:PicoDst"
)
px, py, pz = ak.unzip(
    picodst.arrays(filter_name=["Track/Track.mPMomentum[XYZ]"], entry_stop=100)
)

probable_mass = particle.Particle.from_name("pi+").mass / GeV

pseudojets = ak.zip(
    {"px": px, "py": py, "pz": pz, "mass": probable_mass}, with_name="Momentum4D"
)
good_pseudojets = pseudojets[pseudojets.pt > 0.1]

import fastjet

jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 1.0)

clusterseq = fastjet.ClusterSequence(good_pseudojets, jetdef)
clusterseq.inclusive_jets()

ak.num(good_pseudojets), ak.num(clusterseq.inclusive_jets())